In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("data/train.csv")


In [4]:
na_columns = df.isnull().sum()
na_columns = na_columns[na_columns > 0]

In [7]:
na_columns

X4_sd       16341
X11_sd      16341
X18_sd      16341
X26_sd      16341
X50_sd      16341
X3112_sd    16341
dtype: int64